In [5]:
import random
import base64
import re
import math
import heapq
from collections import Counter

In [6]:
class CanalAdaptativo:
    def __init__(Nodo, inicial, aumento, disminucion):
        Nodo.perdida = inicial
        Nodo.aumento = aumento
        Nodo.disminucion = disminucion

    def transmitir(Nodo):

        if random.random() < Nodo.aumento:
            Nodo.perdida += 0.05
        elif random.random() < Nodo.disminucion:
            Nodo.perdida -= 0.05


        if random.random() < Nodo.perdida:

            return False
        else:

            return True
        
class Huffman:
    def __init__(nodo, simbolo, frecuencia):
        nodo.simbolo = simbolo
        nodo.frecuencia = frecuencia
        nodo.izquierda = None
        nodo.derecha = None

    def __lt__(nodo, otro):
        return nodo.frecuencia < otro.frecuencia

def arbolHuffman(texto):
    if not texto:
        return None 

    conteo = Counter(texto)
    print(conteo)
    list_simbolo = [Huffman(simbolo, frecuencia) for simbolo, frecuencia in conteo.items()]
    heapq.heapify(list_simbolo)

    while len(list_simbolo) > 1:
        izquierda = heapq.heappop(list_simbolo)
        derecha = heapq.heappop(list_simbolo)
        fusion = Huffman(None, izquierda.frecuencia + derecha.frecuencia)
        fusion.izquierda = izquierda
        fusion.derecha = derecha 
        heapq.heappush(list_simbolo, fusion)

    return list_simbolo[0]

def GenCodigosHuffman(Nodo, actual, simbol):
    if Nodo.simbolo is not None:
        simbol[Nodo.simbolo] = actual
        return
    GenCodigosHuffman(Nodo.izquierda, actual + '0', simbol)
    GenCodigosHuffman(Nodo.derecha, actual + '1', simbol)



class ShannonFano:
    def __init__(Nodo) -> None:
        Nodo.simbolo = ''
        Nodo.frecuencia = 0.0
        Nodo.arr = []
        Nodo.alto = 0

p = []

codigosShannon = {}  

# function to find shannon code
def shannon(l, h, p):
	paquete1 = 0; paquete2 = 0; diferencia1 = 0; diferencia2 = 0
	if ((l + 1) == h or l == h or l > h) :
		if (l == h or l > h):
			return
		p[h].alto+=1
		p[h].arr.append(1)
		p[l].alto+=1
		p[l].arr.append(0)
		
		return
	
	else :
		for i in range(l,h):
			paquete1 = paquete1 + p[i].frecuencia
		paquete2 = paquete2 + p[h].frecuencia
		diferencia1 = paquete1 - paquete2
		if (diferencia1 < 0):
			diferencia1 = -diferencia1
		j = 2
		while (j != h - l + 1) :
			k = h - j
			paquete1 = paquete2 = 0
			for i in range(l, k+1):
				paquete1 = paquete1 + p[i].frecuencia
			for i in range(h,k,-1):
				paquete2 = paquete2 + p[i].frecuencia
			diferencia2 = paquete1 - paquete2
			if (diferencia2 < 0):
				diferencia2 = -diferencia2
			if (diferencia2 >= diferencia1):
				break
			diferencia1 = diferencia2
			j+=1
		
		k+=1
		for i in range(l,k+1):
			p[i].alto+=1
			p[i].arr.append(0)
			
		for i in range(k + 1,h+1):
			p[i].alto+=1
			p[i].arr.append(1)
			

		# Invoke shannon function
		shannon(l, k, p)
		shannon(k + 1, h, p)

# Function to sort the symbols
# based on their probability or frequency
def sortByProbability(p):
    p.sort(key=lambda x: x.frecuencia, reverse=True)

# function to display shannon codes
def display(p):
    print("\n\n\n\tSymbol\tProbability\tCode")
    for i in p:
        codigo = ''.join(map(str, i.arr))
        print("\t", i.simbolo, "\t\t", round(i.frecuencia, 5), "\t", codigo)
        codigosShannon[i.simbolo] = codigo

In [7]:
def fuente(filePpath):
    try:
        with open(filePpath, "r", encoding="utf-8") as file:
            text = file.read()
            binarioTexto = ''.join(format(ord(caracter), '08b') for caracter in text)
            Paquete = [binarioTexto[i:i+8] for i in range(0, len(binarioTexto), 8)]
        return Paquete
    except FileNotFoundError:
        print(f"El archivo '{filePpath}' no se encontró.")
        return []
    
def codificador(Paquetes,tamaño,opcion):
    if(opcion == 1):
        raiz = arbolHuffman(Paquetes)
        if raiz is None:
            return "", None
        codigoHuffman = {}
        GenCodigosHuffman(raiz, '', codigoHuffman)
        Empaquetado = [Paquetes[i:i+tamaño] for i in range(0, len(Paquetes), tamaño)]
        EmpaquetadoHuffman = []
        for Paquetes in Empaquetado:
            codificado = ''.join(codigoHuffman[simbolo] for simbolo in Paquetes)
            EmpaquetadoHuffman.append(codificado)
        return EmpaquetadoHuffman, raiz, codigoHuffman
    
    elif(opcion == 2):
        p = [] 
        ascii = list(Paquetes)  
        paquete = []  
        EmpaquetadoShannon = []
        
        for caracter in ascii:
            paquete.append(caracter)
            if len(paquete) == tamaño:
                texto = ''.join(paquete)
                character_frequencies = {caracter: texto.count(caracter) / len(texto) for caracter in set(texto)}
                for caracter, frec in character_frequencies.items():
                    n = ShannonFano()
                    n.simbolo = caracter
                    n.frecuencia = frec
                    p.append(n)
                shannon(0, len(p) - 1, p)
                Empaquetado = ""
                for caracter in texto:
                    codigo = codigosShannon[caracter]
                    Empaquetado += codigo
                EmpaquetadoShannon.append(Empaquetado)
                p.clear()
                paquete.clear()
        if paquete:
            texto = ''.join(paquete)
            character_frequencies = {caracter: texto.count(caracter) / len(texto) for caracter in set(texto)}
            for caracter, frec in character_frequencies.items():
                n = ShannonFano()
                n.simbolo = caracter
                n.frecuencia = frec
                p.append(n)
            shannon(0, len(p) - 1, p)
            Empaquetado = ""
            for caracter in texto:
                codigo = codigosShannon[caracter]
                Empaquetado += codigo
            EmpaquetadoShannon.append(Empaquetado)

        return '|'.join(EmpaquetadoShannon)
    
    elif(opcion == 3):
        simbolos = list(range(256))
        codificado = []
        for simbolo in Paquetes:
            index = simbolos.index(simbolo)
            codificado.append(index)
            del simbolos[index]
            simbolos.insert(0, simbolo)
    
        return codificado
    
    elif(opcion == 4):
        texto = ''.join(chr(int(caracter)) for caracter in Paquetes)
        codificado = []
        i = 0
        while i < len(texto):
            run_length = 1
            while i + 1 < len(texto) and texto[i] == texto[i + 1]:
                run_length += 1
                i += 1
            codificado.append((texto[i], run_length))
            i += 1
        return codificado
    else:
        return Paquetes
    
def sistemaAdaptativo(canales, paquetes):
    intentos = 0
    actual = random.choice(canales)
    recibidos = []
    perdidos = []
    for paquete in paquetes:
        exito = actual.transmitir()
        if exito:
            recibidos.append(paquete)
        else:
            perdidos.append(paquete)
            print("Paquetes perdidos:")
            print(perdidos)
            while perdidos:
                perdido = perdidos.pop(0)
                exito = actual.transmitir()
                intentos += 1
                if exito:
                    recibidos.append(perdido)
        actual = random.choice(canales)
    print("Paquetes recibidos:")
    print(recibidos)
    print(f"Se transmitieron los {len(paquetes)} paquetes con {intentos} intentos.")
    if intentos > 0:
        print("Se requirió retransmisión debido a pérdida de paquetes.")

def decodificador(Paquetes , opcion, raiz):
    if(opcion == 1):
        if raiz is None:
            return ""
        decoficado = []
        Nodo = raiz
        for Empaquetado in Paquetes:
            for bit in Empaquetado:
                if bit == '0':
                    Nodo = Nodo.izquierda
                else:
                    Nodo = Nodo.derecha
                if Nodo.simbolo is not None:
                    decoficado.append(str(Nodo.simbolo))
                    Nodo = raiz
        print(decoficado)
        # Convertir los códigos ASCII a caracteres y unirlos en una cadena
        decoficado = ''.join(chr(int(caracter)) for caracter in decoficado)
        return decoficado
    
    elif(opcion == 2):
        Empaquetado = Paquetes.split('|')  # Dividir en paquetes
        decoficado = ""
        for paquete in Empaquetado:
            paqueteDEC = ""
            codigo = "" 
            for bit in paquete:
                codigo += bit
                for simbolo, simboloCodigo in codigosShannon.items():
                    if codigo == simboloCodigo:
                        paqueteDEC += simbolo
                        codigo = ""
                        break
            decoficado += paqueteDEC
        return decoficado
    
    elif(opcion == 3):
        simbolos = list(range(256))  
        decoficado = []
        for index in Paquetes:
            simbolo = simbolos[index]  
            decoficado.append(simbolo)  
            del simbolos[index]  
            simbolos.insert(0, simbolo) 
        print(decoficado)
        return decoficado
    
    elif(opcion == 4):
        decoficado = ""
        for simbolo, run_length in Paquetes:
            decoficado += simbolo * run_length
        return decoficado
    
    else:
        decoficado = ''.join(chr(int(caracter)) for caracter in Paquetes)
    return decoficado

# Función para reconstruir el libro con los cambios solicitados
def destino(decoded_text, output_file_path):
    decoded_text_str = ''.join(str(item) for item in decoded_text)
    with open(output_file_path, 'w', encoding='utf-8') as f:
        f.write(decoded_text_str)
    print(f"Texto reconstruido guardado en '{output_file_path}'.")

def handshake(Paquetes,tamaño, opcion):
    if opcion == 1:
        canales = [CanalAdaptativo(inicial=0.1, aumento=0.1, disminucion=0.1) for _ in range(6)]
        codificado, root, huffman_codes = codificador(Paquetes,tamaño, opcion)
        sistemaAdaptativo(canales, codificado)
        decoficado = decodificador(codificado , opcion, root)
        print(f"Los códigos Huffman son: {huffman_codes}")
        print(f"Texto original: {Paquetes}")
        print(f"Texto codificado: {codificado}")
        print(f"Texto decodificado: {decoficado}")
        return decoficado
    
    elif opcion == 2:
        canales = [CanalAdaptativo(inicial=0.1, aumento=0.1, disminucion=0.1) for _ in range(6)]
        texto = ''.join(chr(int(caracter)) for caracter in Paquetes)
        frecuenciaCaracter = {simbolo: texto.count(simbolo) / len(texto) for simbolo in set(texto)}
        for simb, frec in frecuenciaCaracter.items():
            n = ShannonFano()
            n.simbolo = simb
            n.frecuencia = frec
            p.append(n)
        sortByProbability(p)
        for i in p:
            i.arr = []
        shannon(0, len(p) - 1, p) 
        display(p)
        codificado = codificador(texto,tamaño, opcion)
        sistemaAdaptativo(canales, codificado)
        decoficado = decodificador(codificado , opcion, None)
        print("\n\n shannon:",codificado)
        print("\n\n shannon:",decoficado)
        return decoficado
    
    elif opcion == 3:
        canales = [CanalAdaptativo(inicial=0.1, aumento=0.1, disminucion=0.1) for _ in range(6)]
        codificado = codificador(Paquetes,tamaño, opcion)
        sistemaAdaptativo(canales, codificado)
        decoficado = decodificador(codificado , opcion, None)
        decoficado = ''.join(chr(int(caracter)) for caracter in decoficado)
        return decoficado
    
    elif opcion == 4:
        canales = [CanalAdaptativo(inicial=0.1, aumento=0.1, disminucion=0.1) for _ in range(6)]
        codificado = codificador(Paquetes,tamaño, opcion)
        sistemaAdaptativo(canales, codificado)
        decoficado = decodificador(codificado, opcion, None)
        return decoficado
    
    else:
        canales = [CanalAdaptativo(inicial=0.1, aumento=0.1, disminucion=0.1) for _ in range(6)]
        codificado = codificador(Paquetes,tamaño, opcion)
        sistemaAdaptativo(canales, codificado)
        decoficado = decodificador(codificado , opcion, None)
        return decoficado

In [8]:
inputFilePath = "test_text.txt"
outputFilePath = "test_text_reconstructed.txt"
Paquetes = fuente(inputFilePath)
ascii = [int(binary, 2) for binary in Paquetes]
book = handshake(ascii,5,3)
destino(book, outputFilePath)

Paquetes perdidos:
[109]
Paquetes perdidos:
[121]
Paquetes perdidos:
[13]
Paquetes perdidos:
[116]
Paquetes perdidos:
[91]
Paquetes perdidos:
[6]
Paquetes perdidos:
[2]
Paquetes perdidos:
[5]
Paquetes perdidos:
[2]
Paquetes perdidos:
[16]
Paquetes perdidos:
[9]
Paquetes perdidos:
[17]
Paquetes perdidos:
[9]
Paquetes perdidos:
[6]
Paquetes perdidos:
[2]
Paquetes perdidos:
[2]
Paquetes perdidos:
[23]
Paquetes perdidos:
[26]
Paquetes perdidos:
[2]
Paquetes perdidos:
[3]
Paquetes perdidos:
[7]
Paquetes perdidos:
[11]
Paquetes perdidos:
[7]
Paquetes perdidos:
[14]
Paquetes perdidos:
[12]
Paquetes perdidos:
[24]
Paquetes perdidos:
[17]
Paquetes perdidos:
[4]
Paquetes perdidos:
[13]
Paquetes perdidos:
[3]
Paquetes perdidos:
[3]
Paquetes perdidos:
[9]
Paquetes perdidos:
[18]
Paquetes perdidos:
[11]
Paquetes perdidos:
[18]
Paquetes perdidos:
[2]
Paquetes perdidos:
[13]
Paquetes perdidos:
[3]
Paquetes perdidos:
[3]
Paquetes perdidos:
[7]
Paquetes perdidos:
[8]
Paquetes recibidos:
[66, 101, 114, 